In [66]:
# Parameters 

application = 'Apple_and_Pears'
varname = 'RAIN_BC'
num_quantiles = 3
target_type = f'cat{num_quantiles}'

In [67]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
%autoreload 2

In [69]:
%matplotlib inline

In [70]:
import sys 
import pathlib

In [71]:
import matplotlib.pyplot as plt 

In [72]:
import numpy as np 
import pandas as pd 
import xarray as xr

In [73]:
np.random.seed(42)

In [74]:
import pycaret

In [75]:
pycaret.__version__

'2.0.0'

In [76]:
from pycaret.classification import *

In [77]:
df_models = models()

In [78]:
df_models

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors.KNeighborsClassifier,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model.SGDClassifier,True
rbfsvm,SVM - Radial Kernel,sklearn.svm.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process.GPC,False
mlp,MLP Classifier,sklearn.neural_network.MLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model.RidgeClassifier,True


In [79]:
HOME = pathlib.Path.home()
CWD = pathlib.Path.cwd()

In [80]:
sys.path.append('/home/nicolasf/research/Smart_Ideas/code/ml4seas')

In [81]:
dpath_indices = pathlib.Path('/media/nicolasf/END19101/data/observational_indices')

In [82]:
indices = pd.read_csv(dpath_indices.joinpath("observational_indices.csv"), index_col=0, parse_dates=True)

In [83]:
indices

,EMI,IOD,NINO34,SAM,SPSD
1979-03-31,0.067251,-0.057314,-0.106222,0.488000,-0.550506
1979-04-30,0.132406,-0.166956,-0.018667,0.644333,-0.253041
1979-05-31,0.141795,0.033624,0.011222,0.767000,0.109477
1979-06-30,0.061373,0.555993,-0.054222,1.034000,-0.016524
1979-07-31,0.020446,0.576727,-0.213889,1.612000,-0.168409
...,...,...,...,...,...
2019-11-30,0.775338,3.802670,0.380667,-0.734000,0.446091
2019-12-31,0.707239,3.106029,0.559444,-1.375000,1.136763
2020-01-31,0.592096,1.886086,0.579667,-1.143667,1.277138
2020-02-29,0.477777,0.418067,0.505000,-0.438667,1.005057


### shift the time variable 

In [105]:
lead = 3

In [106]:
indices.index = indices.index.shift(lead, freq='M')

### Now loads the target 

In [107]:
dpath_target = HOME.joinpath(f"research/Smart_Ideas/outputs/targets/application_cases/{application}/SEASONAL/{varname}")

In [108]:
target = pd.read_csv(dpath_target.joinpath(f"Seasonal_{varname}_mean_anomalies_and_Q{num_quantiles}_categories.csv"), index_col=0, parse_dates=True) 

In [109]:
target

,cat3,anomalies
time,,
1979-03-31,3,2.044368
1979-04-30,3,1.925033
1979-05-31,3,1.502712
1979-06-30,1,-0.938936
1979-07-31,1,-0.900086
...,...,...
2019-08-31,2,-0.440571
2019-09-30,2,-0.384193
2019-10-31,3,0.609661


In [110]:
target = target.loc[:,[target_type]]

In [111]:
data = pd.concat([indices, target], axis=1)

In [112]:
data.head()

,EMI,IOD,NINO34,SAM,SPSD,cat3
1979-03-31,NaN,NaN,NaN,NaN,NaN,3.0
1979-04-30,NaN,NaN,NaN,NaN,NaN,3.0
1979-05-31,NaN,NaN,NaN,NaN,NaN,3.0
1979-06-30,0.067251,-0.057314,-0.106222,0.488000,-0.550506,1.0
1979-07-31,0.132406,-0.166956,-0.018667,0.644333,-0.253041,1.0


In [113]:
data.tail()

,EMI,IOD,NINO34,SAM,SPSD,cat3
2020-02-29,0.775338,3.802670,0.380667,-0.734000,0.446091,NaN
2020-03-31,0.707239,3.106029,0.559444,-1.375000,1.136763,NaN
2020-04-30,0.592096,1.886086,0.579667,-1.143667,1.277138,NaN
2020-05-31,0.477777,0.418067,0.505000,-0.438667,1.005057,NaN
2020-06-30,0.423151,-0.062580,0.493778,0.490000,0.560904,NaN


In [114]:
data = data.dropna(axis=0)

In [115]:
data

,EMI,IOD,NINO34,SAM,SPSD,cat3
1979-06-30,0.067251,-0.057314,-0.106222,0.488000,-0.550506,1.0
1979-07-31,0.132406,-0.166956,-0.018667,0.644333,-0.253041,1.0
1979-08-31,0.141795,0.033624,0.011222,0.767000,0.109477,2.0
1979-09-30,0.061373,0.555993,-0.054222,1.034000,-0.016524,3.0
1979-10-31,0.020446,0.576727,-0.213889,1.612000,-0.168409,3.0
...,...,...,...,...,...,...
2019-08-31,0.387744,0.961243,0.751222,0.472000,2.707132,2.0
2019-09-30,0.401512,1.663473,0.662444,0.712000,2.656905,2.0
2019-10-31,0.542545,2.102538,0.572778,0.470000,2.276956,3.0
2019-11-30,0.733343,2.604696,0.372111,-0.001667,1.733595,3.0


In [116]:
silent = True

In [117]:
log_name = f"PYCARET_exp_obs_indices_target_{varname}_{target_type}"

In [118]:
print(f"experiment on MLFlow in {log_name}")

experiment on MLFlow in PYCARET_exp_obs_indices_target_RAIN_BC_cat3


In [119]:
exp_clf = setup(data = data, \
                target = f'{target_type}', \
                session_id=123, \
                log_experiment=True, \
                experiment_name=log_name, \
                normalize = True, \
                transformation = False, \
                silent=silent)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Multiclass
2,Label Encoded,None
3,Original Data,"(487, 6)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [120]:
top5 = compare_models(turbo=True, n_select=5) 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extra Trees Classifier,0.4706,0.0000,0.4697,0.4858,0.4676,0.2050,0.2109,0.0990
1,Random Forest Classifier,0.4441,0.0000,0.4437,0.4644,0.4402,0.1653,0.1702,0.0194
2,Ada Boost Classifier,0.4412,0.0000,0.4424,0.4519,0.4384,0.1625,0.1665,0.0576
3,CatBoost Classifier,0.4382,0.0000,0.4371,0.4459,0.4344,0.1569,0.1594,1.1817
4,Light Gradient Boosting Machine,0.4294,0.0000,0.4293,0.4409,0.4229,0.1430,0.1473,0.0579
5,K Neighbors Classifier,0.4265,0.0000,0.4255,0.4409,0.4171,0.1391,0.1456,0.0017
6,Logistic Regression,0.4088,0.0000,0.4078,0.4096,0.4006,0.1118,0.1144,0.0062
7,Ridge Classifier,0.4088,0.0000,0.4078,0.4073,0.3994,0.1116,0.1141,0.0033
8,Extreme Gradient Boosting,0.4088,0.0000,0.4081,0.4066,0.4010,0.1120,0.1146,0.0747
9,Naive Bayes,0.4059,0.0000,0.4063,0.4031,0.3983,0.1100,0.1108,0.0022


In [121]:
table_top5 = pull()

In [122]:
table_top5

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extra Trees Classifier,0.4706,0.0,0.4697,0.4858,0.4676,0.2050,0.2109,0.0990
1,Random Forest Classifier,0.4441,0.0,0.4437,0.4644,0.4402,0.1653,0.1702,0.0194
2,Ada Boost Classifier,0.4412,0.0,0.4424,0.4519,0.4384,0.1625,0.1665,0.0576
3,CatBoost Classifier,0.4382,0.0,0.4371,0.4459,0.4344,0.1569,0.1594,1.1817
4,Light Gradient Boosting Machine,0.4294,0.0,0.4293,0.4409,0.4229,0.1430,0.1473,0.0579
5,K Neighbors Classifier,0.4265,0.0,0.4255,0.4409,0.4171,0.1391,0.1456,0.0017
6,Logistic Regression,0.4088,0.0,0.4078,0.4096,0.4006,0.1118,0.1144,0.0062
7,Ridge Classifier,0.4088,0.0,0.4078,0.4073,0.3994,0.1116,0.1141,0.0033
8,Extreme Gradient Boosting,0.4088,0.0,0.4081,0.4066,0.4010,0.1120,0.1146,0.0747
9,Naive Bayes,0.4059,0.0,0.4063,0.4031,0.3983,0.1100,0.1108,0.0022


boosting (CatBoost Classifier, Extreme Gradient Boosting, Extra Trees Classifier, Light Gradient Boosting Machine, Gradient Boosting Classifier) 
all seem to perform quite well ....

In [123]:
top5[0:1]

[ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=123, verbose=0,
                      warm_start=False)]

### blend the best models 

In [124]:
blender = blend_models(estimator_list=top5[0:3], method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5588,0.0000,0.5631,0.5618,0.5475,0.3411,0.3483
1,0.5882,0.0000,0.5884,0.6175,0.5863,0.3826,0.3935
2,0.4118,0.0000,0.4116,0.4326,0.4187,0.1180,0.1191
3,0.5294,0.0000,0.5278,0.5588,0.5349,0.2898,0.2949
4,0.5000,0.0000,0.4949,0.4907,0.4948,0.2484,0.2487
5,0.3824,0.0000,0.3864,0.3789,0.3737,0.0775,0.0788
6,0.4412,0.0000,0.4444,0.4641,0.4336,0.1654,0.1739
7,0.4706,0.0000,0.4672,0.4739,0.4680,0.2021,0.2040
8,0.4412,0.0000,0.4369,0.4442,0.4349,0.1599,0.1627
9,0.5588,0.0000,0.5631,0.5739,0.5622,0.3394,0.3425


### tune the best model 